## 1. Simulate standard example of two particles with an unstable algorithm

In [1]:

import numpy as np 
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path+"/openmd")

from SimulatorLJ import SimulatorLJ as Simulator


gamma = 0.1
dt = 0.001
coordinates = np.array([[0,1,0],[0,2,0]])
velocities = gamma*np.array([[0,1,0],[0,-1,0]])
constants = [1.0, 0.5]
box_len = 6
sim = Simulator(
    path = r"C:\Users\ZhenAK97\Desktop",
    title = "test_1",
    mass = 1,
    sim_time= 10 ,
    time_step = dt,
    initial_values = [coordinates,velocities],
    box_length = box_len,
    force=None,
    force_constants=constants,
    integrator=None,   
    periodic=True,
)

positions, velocities =sim.simulate()

c:\Users\ZhenAK97\Documents\GitHub\openmd


c:\Users\ZhenAK97\Documents\GitHub\openmd/openmd\SimulatorLJ.py:182: RuntimeWarning: divide by zero encountered in true_divide
  force_t = -48 * epsilon * np.power(sigma, 12) / np.power(
c:\Users\ZhenAK97\Documents\GitHub\openmd/openmd\SimulatorLJ.py:184: RuntimeWarning: divide by zero encountered in true_divide
  ) - 24 * epsilon * np.power(sigma, 7) / np.power(difference, 7)
c:\Users\ZhenAK97\Documents\GitHub\openmd/openmd\SimulatorLJ.py:223: RuntimeWarning: invalid value encountered in subtract
  difference = positions - particle
c:\Users\ZhenAK97\Documents\GitHub\openmd/openmd\SimulatorLJ.py:207: RuntimeWarning: divide by zero encountered in true_divide
  potential_t = 4 * epsilon * np.power(sigma, 12) / np.power(
c:\Users\ZhenAK97\Documents\GitHub\openmd/openmd\SimulatorLJ.py:209: RuntimeWarning: divide by zero encountered in true_divide
  ) - 4 * epsilon * np.power(sigma, 6) / np.power(difference, 6)
c:\Users\ZhenAK97\Documents\GitHub\openmd/openmd\SimulatorLJ.py:207: RuntimeWarn

(10000,)
(10000,)
(10000,)


In [ ]:
import matplotlib.pyplot as plt 
plt.plot(positions[0,:,:].T)

# 2. Stable algorithm

In [ ]:

def force_lj_fast( positions, constants, box_length):
    """
    :param positions:
    :type positions:
    :param constants:
    :type constants:
    :param box_length:
    :type box_length:
    :return:
    :rtype:
    """
    
    epsilon, sigma = constants
    num = positions.shape[0]
    force = np.zeros((num, num, 3))
    for i in range(0, num - 1):
        for j in range(i + 1, num):
            difference = positions[j, :] - positions[i, :]
            distance = np.linalg.norm(difference)
            #print("distance is", distance)
            if distance > box_length / 2:
                distance -= box_length / 2
            elif distance <= -box_length / 2:
                distance += box_length / 2
            if distance == 0: 
                force[i,j] = 0 
                force[j,i] = 0
                break
            #print("distance is", distance)
            lj_part = (sigma / distance) ** 6
            #print("dist, sig", distance, sigma)
            #print(sigma/distance)
            #print(0.01/3)
            #print("lj",lj_part)
            lj_part_two = lj_part ** 2
            factor = 24 * epsilon
            factor_two = 2 * factor
            force[i, j, :] = (factor_two * lj_part_two - factor * lj_part) * (difference/distance)
            force[j, i, :] -= force[i, j, :]
    #print(np.sum(force, axis=1))
    return np.sum(force, axis=1)

In [ ]:
sim.force = force_lj_fast
sim.force

In [ ]:
gamma = 0.1
dt = 0.001
coordinates = np.array([[0,1,0],[0,2,0]])
velocities = gamma*np.array([[0,1,0],[0,-1,0]])
constants = [1.0, 0.5]
box_len = 6
sim = Simulator(
    mass = 4,
    sim_time= 40000*dt ,
    time_step = dt,
    initial_values = [coordinates,velocities],
    box_length = box_len,
    force=None,
    force_constants=constants,
    integrator=None,   
    periodic=True,
)
sim.force = force_lj_fast
sim.force

positions, velocities =sim.simulate()

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(positions[1, 1,:].T)

In [ ]:
def energy_lj(positions, constants, box_length):
    """
    """
    for i in range(0, num - 1):
        for j in range(i + 1, num):
            difference = positions[j, :] - positions[i, :]
            distance = np.linalg.norm(difference)
            if distance > box_length / 2:
                distance -= box_length / 2
            elif distance <= -box_length / 2:
                distance += box_length / 2


def energy_lj_fast(positions, constants, box_length):
    """Fast energy calculation, e.g. for MC initialization

    Args:
        positions ([type]): [description]
        constants ([type]): [description]
        box_length ([type]): [description]

    Returns:
        [type]: [description]
    """
    epsilon, sigma = constants
    separations = positions[:, None, :] - positions
    # check periodic boundary conditions
    separations[separations > box_length / 2] -= box_length
    separations[separations <= -box_length / 2] += box_length

    # calculate NxN matrix with distances |r[i] - r[j]|
    # set zero values to None for calculation of acceleration

    distances = np.linalg.norm(separations, axis=-1)
    distances[distances == 0] = None

    # calculate potential energy for Lennard Jones potential
    e_pot = (
        4
        * epsilon
        * (np.power((distances / sigma), -12) - np.power((distances / sigma), -6))
    )
    e_pot[np.isnan(e_pot)] = 0
    return np.sum(e_pot)


def kinetic_energy(V, M):
    """Calculates the kinetic energy 

    Args:
        V ([type]): [description]
        M ([type]): [description]
    """

    # normed_vel =np.sum(0.5 * M*V*V,axis=0)
    # assert 1 == 2, str(len(V))
    e_kin = 0
    for i in range(len(V)):
        e_kin += 0.5 * M * np.sum(np.square(V[i, :]))
    return e_kin


def energy_gravity(positions, constants, box_length):
    """Potential energy calculation for the gravitational potential

    Args:
        positions ([type]): [description]
        constants ([type]): [description]
        box_length ([type]): [description]
    """
    G = constants[0]
    separations = positions[:, None, :] - positions

    # check periodic boundary conditions
    separations[separations > box_length / 2] -= box_length
    separations[separations <= -box_length / 2] += box_length

    # calculate NxN matrix with distances |r[i] - r[j]|
    # set zero values to None for calculation of acceleration

    distances = np.linalg.norm(separations, axis=-1)
    distances[distances == 0] = None

    # calculate potential energy for Lennard Jones potential
    e_pot = -1 * G * 1 / distances
    e_pot[np.isnan(e_pot)] = 0

    return np.sum(e_pot)

In [ ]:
energy =[]
e_kin = []
for i in range(positions.shape[2]): 
    energy.append(energy_lj_fast(positions[:,:,i], constants=constants, box_length=box_len))
    e_kin.append(kinetic_energy(velocities[:, :, i], 1))
    

In [ ]:
plt.plot(energy)

In [ ]:
plt.plot(e_kin)

In [ ]:
plt.plot(sum(energy, e_kin))